In [1]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
spark = SparkSession.builder.appName("BigDataFinalProject")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .config("spark.driver.memory", "20g")\
    .getOrCreate()
plt.style.use('ggplot')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/06 05:11:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# %env HADOOP_USER_NAME=hadoopuser

In [5]:

# val parqDF = spark.read.parquet("hdfs://nn1home:8020/people.parquet")
# Using spark.write.parquet() function we can write Spark DataFrame to Parquet file, and parquet() function is provided in DataFrameWriter class. 


# 
# df = spark.read.format("parquet").load(paths[0])
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType
# data2 = [("James","","Smith","36636","M",3000),
#     ("Michael","Rose","","40288","M",4000),
#     ("Robert","","Williams","42114","M",4000),
#     ("Maria","Anne","Jones","39192","F",4000),
#     ("Jen","Mary","Brown","","F",-1)
#   ]

# schema = StructType([ \
#     StructField("firstname",StringType(),True), \
#     StructField("middlename",StringType(),True), \
#     StructField("lastname",StringType(),True), \
#     StructField("id", StringType(), True), \
#     StructField("gender", StringType(), True), \
#     StructField("salary", IntegerType(), True) \
#   ])
 
# df = spark.createDataFrame(data=data2,schema=schema)

# df.write.parquet("hdfs://20.121.35.246:9000/user/root/test3.parquet")
# a = spark.sparkContext.textFile("")

# parqDF = spark.read.parquet("hdfs://20.121.35.246:9000/user/root/test3.parquet")
# parqDF.printSchema()

In [6]:
# df.count()

In [7]:
# parqDF.count()

In [8]:
# a

/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2019-02.parquet

In [9]:
spark

In [10]:
# data = spark.read.parquet('/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_*')
# import glob
# paths=glob.glob("/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_*")
# print(paths)

# data = spark.read.format("parquet").load(paths)
data = spark.read.parquet("hdfs://20.121.35.246:9000/nyc_trips")

['/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2019-04.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2020-11.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2021-02.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2019-03.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2020-10.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2020-06.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2020-09.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2022-09.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2021-10.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2021-05.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2020-04.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2022-11.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2020-05.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2019-07.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2019-08.parquet', '/kaggle/input/nyc-fhvhv-data/fhvhv_tripdata_2022-01.p

In [11]:
data.createOrReplaceTempView('TAXI')

In [12]:
data

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, originating_base_num: string, request_datetime: timestamp_ntz, on_scene_datetime: timestamp_ntz, pickup_datetime: timestamp_ntz, dropoff_datetime: timestamp_ntz, PULocationID: bigint, DOLocationID: bigint, trip_miles: double, trip_time: bigint, base_passenger_fare: double, tolls: double, bcf: double, sales_tax: double, congestion_surcharge: double, airport_fee: int, tips: double, driver_pay: double, shared_request_flag: string, shared_match_flag: string, access_a_ride_flag: string, wav_request_flag: string, wav_match_flag: int]

## Analisis de datos

In [13]:
data.count()

745287023

745287023

In [31]:
data = data.drop("wav_request_flag").drop("wav_match_flag").drop("airport_fee")
data.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nullable = true)
 |-- dayofwee

In [15]:
data.select(F.min('tips').alias('min'), F.max('tips').alias('max')).show()

+---+------+
|min|   max|
+---+------+
|0.0|1000.0|
+---+------+



In [16]:
data.select('tips').describe().toPandas()

,summary,tips
0,count,745287023
1,mean,0.7621059631141537
2,stddev,2.5219715698808893
3,min,0.0
4,max,1000.0


In [17]:
req = """
select * from TAXI
where tips<500
"""
data = spark.sql(req)
data.select('tips').describe().toPandas()

,summary,tips
0,count,745286977
1,mean,0.762074392801171
2,stddev,2.5187022761997007
3,min,0.0
4,max,497.0


In [32]:
data = data.withColumn('dayofweek', F.dayofweek(data['pickup_datetime']))
data = data.withColumn('hour', F.hour(data['pickup_datetime']))

# peak hours, Monday-Friday between 4PM-8PM
data = data.withColumn('peak_hours', F.when(
    (data['hour']>=16) & (data['hour']<20) & (data['dayofweek']!=7)&(data['dayofweek']!=0), 1).otherwise(0))

# night hours, $0.5 of additional surcharge between 8PM - 6AM.
data = data.withColumn('night_time', F.when(
    (data['hour']>=20) | (data['hour']<6), 1).otherwise(0))

# Modelamiento

In [33]:
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed=23784)

In [34]:
continuous_variables = ['dayofweek',
                        'hour',
                        'peak_hours',
                        'night_time',
                        'trip_time',
                        'PULocationID',
                        'DOLocationID']

In [35]:
assembler = VectorAssembler(
    inputCols=continuous_variables,
    outputCol='features')

# Entrenamiento

In [36]:
trainingData = assembler.setHandleInvalid("skip").transform(trainingData)
testData = assembler.setHandleInvalid("skip").transform(testData)

# Decision Tree Regressor

In [ ]:
dt = DecisionTreeRegressor(featuresCol='features', labelCol='tips')

# Model fitting
model = dt.fit(trainingData)

# Get oredictions
predictions = model.transform(testData)

In [ ]:
print(model)

# Evaluacion

In [ ]:
evaluator = RegressionEvaluator(
    labelCol='tips',  predictionCol="prediction", metricName="rmse")

In [ ]:
rmse_train = evaluator.evaluate(model.transform(trainingData))
print("Train Root Mean Squared Error (RMSE) on test data = %g" % rmse_train)

In [ ]:
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

In [ ]:
testData.select(F.mean(testData['tips'])).show()

In [ ]:
avr = testData.agg(F.mean(testData['tips']).alias("mean")).collect()[0]["mean"]
testData_upd = testData.withColumn('subt',((testData['tips'] - avr)**2)**0.5)

# RMSE for naive hypothesis
testData_upd.agg(F.mean(testData_upd['subt'])).show()

In [ ]:
gbt = GBTRegressor(featuresCol='features', labelCol='tips')
m = gbt.fit(trainingData)
predictions = m.transform(testData)

#evaluete model
evaluator = RegressionEvaluator(labelCol='tips', predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("GBT model. Root Mean Squared Error (RMSE) on test data = %g" % rmse)
m.featureImportances